![Egeria Logo](https://raw.githubusercontent.com/odpi/egeria/master/assets/img/ODPi_Egeria_Logo_color.png)

### Egeria Hands-On Lab
# Welcome to the Metadata Representation Demo Lab

## Introduction

Egeria is an open source project that provides open standards and implementation libraries to connect tools, catalogs and platforms together so they can share information (called metadata) about data and the technology that supports it.

In this hands-on lab you will set up an open metadata repository loaded with metadata and then issue queries to retrieve different types of metadata to understand how it represents popular technologies.

The Egeria team use the personas and scenarios from the fictitious company called [Coco Pharmaceuticals](https://egeria-project.org/practices/coco-pharmaceuticals/).
As part of the huge business transformation that Coco Pharmaceuticals has embarked on, they
have decided to use Egeria to manage their metadata across the enterprise.

## The Scenario

[Polly Tasker](https://egeria-project.org/practices/coco-pharmaceuticals/personas/polly-tasker/),
the lead of IT development asked [Peter Profile](https://egeria-project.org/practices/coco-pharmaceuticals/personas/peter-profile/) and [Erin Overview](https://egeria-project.org/practices/coco-pharmaceuticals/personas/erin-overview/) from the governance team to give a series of talks and a demos about Egeria to her team, who will be building integration code to connect various tools and applications into Egeria.


<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/polly-tasker.png">
  <figcaption style="margin-left: 7%;"><strong>Polly Tasker</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/peter-profile.png">
  <figcaption style="margin-left: 7%;"><strong>Peter Profile</strong></figcaption>
</figure>

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="https://raw.githubusercontent.com/odpi/egeria-docs/main/site/docs/practices/coco-pharmaceuticals/personas/erin-overview.png">
  <figcaption style="margin-left: 7%;"><strong>Erin Overview</strong></figcaption>
</figure>


Peter and Erin plan to use this session to show how Egeria's open types can representen the different types of data sources used in Coco Pharmaceuticals.  They have been asked to set up the demo on the **Development [OMAG Server Platform](https://egeria-project.org/concepts/omag-server-platform/)** so that the team can run it themselves at a later time.

## Configuring the metadata catalog

Peter first checks that the Development OMAG Server Platform is running.

----

In [ ]:
%run ../common/common-functions.ipynb

print(" ")
result = checkServerPlatform(devPlatformName, devPlatformURL)
print(" ")


----

If the platform is not running, follow [this link to set up and run the platform](https://egeria-project.org/education/open-metadata-labs/overview/).  Then re-run the previous step to ensure the platform is started.

----

Peter sets up a new [metadata access store](https://egeria-project.org/concepts/metadata-access-store/) with sample metadata pre-loaded through an open metadata archive file.  The metadata they are using comes from retail data they they have been using in testing.  There is no real pharmaceutical data used in the demos.

<figure style="margin-left: 7%; display:inline-block;">  
  <img src="../images/simple-catalog.png">
</figure>


----

In [ ]:

def configureSimpleCatalog(mdrServerName, mdrRepositoryType, metadataCollectionId, metadataCollectionName, archiveFileNames):
    eventBusURLroot   = os.environ.get('eventBusURLroot', 'localhost:9092')
    eventBusBody      = {
        "producer": {
             "bootstrap.servers": eventBusURLroot
         },
         "consumer": {
             "bootstrap.servers": eventBusURLroot
         }
    }
    print("Configuring " + mdrServerName + "...")
    configurePlatformURL(devPlatformURL, adminUserId, mdrServerName, devPlatformURL)
    configureMaxPageSize(devPlatformURL, adminUserId, mdrServerName, '600')
    clearServerType(devPlatformURL, adminUserId, mdrServerName)
    configureOwningOrganization(devPlatformURL, adminUserId, mdrServerName, "Coco Pharmaceuticals Dev Systems")
    configureUserId(devPlatformURL, adminUserId, mdrServerName, "simpleMDSnpa")
    configurePassword(devPlatformURL, adminUserId, mdrServerName, "simpleMDSpassw0rd")
    configureEventBus(devPlatformURL, adminUserId, mdrServerName, eventBusBody)
    configureMetadataRepository(devPlatformURL, adminUserId, mdrServerName, mdrRepositoryType)
    configureDescriptiveName(devPlatformURL, adminUserId, mdrServerName, metadataCollectionName)
    configureMetadataCollectionId(devPlatformURL, adminUserId, mdrServerName, metadataCollectionId)
    removeAllStartupArchive(devPlatformURL, adminUserId, mdrServerName)
    if archiveFileNames:
        for x in range(len(archiveFileNames)):
            addStartupArchive(devPlatformURL, adminUserId, mdrServerName, archiveFileNames[x])
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'data-manager', {})
    configureAccessService(devPlatformURL, adminUserId, mdrServerName, 'asset-owner', {})


mdrServerName = "SimpleCatalog"

configureSimpleCatalog(mdrServerName, "in-memory-repository", "e915f2fa-aaac-4396-8bde-bcd65e642b1d", mdrServerName, ["content-packs/SimpleAPICatalog.omarchive", "content-packs/SimpleDataCatalog.omarchive", "content-packs/SimpleEventCatalog.omarchive","content-packs/SimpleGovernanceCatalog.omarchive"])

print("\nDone.")


----

Now that the server is configured, Peter starts it up ...

----

In [ ]:
activatePlatform(devPlatformName, devPlatformURL, [mdrServerName])

print("\nDone.")


----

Peter checks that the server is running on the platform ...

----

In [ ]:
queryActiveServers(devPlatformName, devPlatformURL)

----

### Searching for Assets

[Assets](https://egeria-project.org/concepts/asset/) describe the digital resources (data sources, systems, processes, ...) that are of value to the organization.
Erin begins with a query to show the different types of asset loaded into the catalog.  There are four assets returned:

* BRANCH - a database
* RETAILSCHEMA - a database schema within BRANCH
* Customer API - an API for querying customer details
* CustomerChangeTopic - a topic used to publish information about changes to customer details

----

The properties of each asset displayed are:

<center>
    <img src="../images/simple-catalog-demo-asset-properties.png">
</center>

| Properties | Description  |
| :----------- | :------------ |
| Display Name | A simple name for display. |
| Unique Identifer (GUID) | Globally unique identifier assigned by the repository. |
| Unique Name  | A unique name assigned by the creator of the metadata. Known as the *qualifiedName*.|
| Type  | The open metadata type ([DeployedDatabaseSchema](https://egeria-project.org/types/2/0224-Databases/) in this example) define the specialized properties (attributes) that can be added to a metadata element of the the type. |
| Super Types | The types that DeployedDatabaseSchema, in this example, inherits from.  The attributes allowed in a metadata element are accumulated from its type and all of its super types. |

----


In [ ]:
assetOwnerPrintAssets(mdrServerName, devPlatformName, devPlatformURL, erinsUserId, ".*")

Erin focusses on each type of asset in turn ...

### Representing databases

Assets can be linked to many other types of metadata elements to provide more details about its characteristics and how it should be managed.
For example, data governance ensures that the data values used by an organization are fit for purpose.
To support data governance, open metadata must capture how the data values are organized in the data stores.
This organization is called the *schema*.  There are different types of schema that represent the different types of data structures within data assets.
Therefore, the schema attached to the asset describes both its type, and how the data values are organized within it.

This diagram shows how database are described in open metadata.  It shows a database called *BRANCH* that has one database schema called *RETAILSCHEMA* that includes a *CUSTOMER* table with four columns: CUSTID, CUSTNAME, CUSTSTATUS, CUSTCARD.

The metadata elements for *BRANCH* and *RETAILSCHEMA* are shown in yellow to indicate that they are *Assets*. The values shown in blue are *SchemaTypes*.  They describe the structure/format of data.  The values shown in pink are *SchemaAttributes*, which are here the data values.


<center>
    <img src="../images/simple-data-catalog.png">
</center>

[Lemmie Stage](https://egeria-project.org/practices/coco-pharmaceuticals/personas/lemmie-stage/) asked why there was a separation between a *SchemaType* and a *SchemaAttrbute*.  Erin tries to map this to programming and suggests he thinks of the schema attributes as the program variables, and the schema types as data types.

[Bob Nitter](https://egeria-project.org/practices/coco-pharmaceuticals/personas/bob-nitter/), asked why the database schema *RETAILSCHEMA* is an *Asset* rather than *SchemaType* which would make more sense given its name.  Erin agrees that the name is confusing.  However, it is important to focus on how databases are used.  Often applications are only dependent on a particular database schema.  They are not concerned with the other database schemas that happen to be deployed on the database.  In fact, the database schemas deployed to a database may be somewhat arbitrary.  Therefore, representing the database schema as the application's data asset is a more accurate reflection of the data use of the application.


----

Erin issues a query that returns more details about *BRANCH Database*.

----

In [ ]:
printSelectiveAssetUniverse(mdrServerName, devPlatformName, devPlatformURL, 'data-manager', erinsUserId, "076b4498-a099-4c6b-9243-73e816b28504", False, True)

----

She points out that *RETAILSCHEMA* is connected to *BRANCH* using the [DataContentForDataSet](https://egeria-project.org/types/2/0210-Data-Stores/) relationship.  This indicates that the data returned for *RETAILSCHEMA* is stored in *BRANCH*.  This means that *RETAILSCHEMA* shows the data used by a particular application and the *DataContentForDataSet* relationship shows where the database schema is deployed to.

Erin highlights the type hierarchy for [Database](https://egeria-project.org/types/2/0224-Databases/) and [DeployedDatabaseSchema](https://egeria-project.org/types/2/0224-Databases/).

<center>
    <img src="../images/database-type-hierarchy.png">
</center>

They are both assets, but Database is a DataStore (physical data store) and DeployedDatabaseSchema is a DataSet (logical collection of data).  Typically, business user's are more interested in DataSets, whereas governance processes such back-up focus on the DataStores.

----

Erin then displays detailed information about *RETAILSCHEMA*.  It has a *schemaType* of type [RelationalDBSchemaType](https://egeria-project.org/types/5/0534-Relational-Schemas/) which means it is a relational database schema consisting of tables, views and columns.

The [Anchors](https://egeria-project.org/types/0/0010-Base-Model/#anchors) classification ensures that the the schema elements are deleted when their *anchor* element (ie *RETAILSCHEMA*) is deleted.  It prevents fragments of schema and other dependent elements being left in the open metadata ecosystem when their asset is deleted.

There are two levels of schema attributes nested in the RelationalDBSchemaType.  The outer level contains the tables (schema attributes of type [RelationalTable](https://egeria-project.org/types/5/0534-Relational-Schemas/)) and nested inside are the columns (schema attributes of type [RelationalColumn](https://egeria-project.org/types/5/0534-Relational-Schemas/)).

The [TypeEmbeddedAttribute](https://egeria-project.org/types/5/0505-Schema-Attributes/#typeembeddedattribute-classification) describes the type of the attribute.  So the Relational Table *CUSTOMER* has a type of *RelationalTableType* and the RelationalColumns: CUSTID, CUSTNAME, CUSTSTATUS, CUSTCARD each have a type of [PrimitiveSchemaType](https://egeria-project.org/types/5/0501-Schema-Elements/#simpleschematype).   The specific data type is *CHAR*.

----

In [ ]:
printSelectiveAssetUniverse(mdrServerName, devPlatformName, devPlatformURL, 'data-manager', erinsUserId, "4782e08b-043c-4017-9b2f-d63163f67fd8", False, True)

----

Relationships are two-way, so Erin points out that *BRANCH* is connected *RETAILSCHEMA* using the *DataContentForDataSet* relationship.  The *attributeName* identifies the role of each asset in the relationship.

----

### Topics and Events

Erin now moves onto event-based data resources.  In this simple demo, there is an Apache Kafka topic in the catalog.
A topic is an data resource that delivers events posted by producers to one or more subscribing consumers.
The asset that represents it in the catalog is of type [KafkaTopic](https://egeria-project.org/types/2/0223-Events-and-Logs/#kafkatopic).

The structure of an event is represented as an [EventType](https://egeria-project.org/types/5/0535-Event-Schemas/#eventtype) *SchemaType*.  The data values within it are represented as [EventSchemaAttribute](https://egeria-project.org/types/5/0535-Event-Schemas/#eventschemaattribute) *SchemaAttributes*.

It is possible that the topic supports multiple event types.  If that is the case, the topic asset is connected to an [EventTypeList](https://egeria-project.org/types/5/0535-Event-Schemas/#eventtypelist) which has a [SchemaOption](https://egeria-project.org/types/5/0501-Schema-Elements/#schematypechoice-and-schematypeoption) relationship to each of the possible EventTypes.

In the catalog the Apache Kafka topic is called *CustomerChangeTopic*.  It has an EventTypeList called *CustomerChangeTopic Event Type List* with one EventType called *New Customer Status Event Type*.  The EventType has three data fields in it called: *customerId*, *customerName* and *customerStatus*.

<center>
    <img src="../images/simple-event-catalog.png">
</center>

----

This is the detail of the Kafka Topic.  There are no related assets, but the *SchemaType* is an EventTypeList with one embedded EventType *SchemaType*.  Nested in the EventType are three EventSchemaAttribute *SchemaAttributes* called *customerStatus*, *customerId* and *customerName*.
    
----

In [ ]:
printSelectiveAssetUniverse(mdrServerName, devPlatformName, devPlatformURL, 'data-manager', erinsUserId, "baded87e-7fe2-4d50-963c-b87178afc452", False, True)

----

### APIs

The API in the catalog provides information about an API and its schema.  The schema describes the interface of the API and follows the definition of the [OpenAPI Specification](https://en.wikipedia.org/wiki/OpenAPI_Specification) standard.

The root *SchemaType* is an [APISchemaType](https://egeria-project.org/types/5/0536-API-Schemas/) called *CustomerAPI Specification*.  It shows one API operation called *getCustomer*.  This operation takes a *customerNo* string as an input parameter and returns *customerNo*, *customerName* and *customerCardId* strings in the response.

<center>
    <img src="../images/simple-api-catalog.png">
</center>


----

This is the detail of the API.
    
----

In [ ]:
printSelectiveAssetUniverse(mdrServerName, devPlatformName, devPlatformURL, 'data-manager', erinsUserId, "81ff5b28-6e24-4946-b5f5-90fdb277bab8", False, True)

----

## Summary

This lab has shown how databases, events and APIs are represented in the open metadata ecosystem.  There is more information in Egeria's documentation, specifically on [Assets](https://egeria-project.org/concepts/asset/) and the different [schema representations](https://egeria-project.org/concepts/schema/).

## Where to next?

* The [Simple Cohort Demo](simple-cohort-demo.ipynb) lab shows how an [Open Metadata Repository Cohort](https://egeria-project.org/features/cohort-operation/overview/) works.
* The [Governance server operation lab](governance-server-operation.ipynb) shows how [integration connectors](https://egeria-project.org/concepts/integration-connector/) and [governance action services](https://egeria-project.org/concepts/governance-action-service/) run on Egeria's [Governance Servers](https://egeria-project.org/concepts/governance-server/).
* The [Building a data catalog lab](../asset-management-labs/building-a-data-catalog.ipynb) shows how files are represented plus examples of governance metadata that can be added to assets.


----